<a href="https://colab.research.google.com/github/sarvanChamp/Iris-/blob/main/RDDHandsOn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,to_date



In [ ]:
from google.colab import files
uploaded=files.upload()

Saving sales.csv to sales.csv


In [ ]:
spark=SparkSession.builder.appName("ETL_Sample").getOrCreate()

df=spark.read.option("header", True).option("inferschema", True).csv("sales.csv")

print("Column Names", df.columns)

df.show()

Column Names ['order_id', 'customer_id', 'product', 'price', 'quantity', 'order_date']
+--------+-----------+----------+-----+--------+----------+
|order_id|customer_id|   product|price|quantity|order_date|
+--------+-----------+----------+-----+--------+----------+
|    1001|       C001|    Laptop|70000|       1|01-06-2024|
|    1002|       C002|Smartphone|30000|       2|02-06-2024|
|    1003|       C001|     Mouse|  500|       3|03-06-2024|
|    1004|       C003|  Keyboard| 1500|       1|04-06-2024|
|    1005|       C002|   Monitor|12000|       1|05-06-2024|
|    1006|       C004|      NULL|45000|       1|06-06-2024|
|    1007|       C005|    Tablet|25000|    NULL|07-06-2024|
|    1008|       C001|    Laptop|70000|       1|08-06-2024|
|    1009|       C006|Smartwatch| 5000|       1|      NULL|
|    1010|       C007|     Mouse|  500|       2|10-06-2024|
+--------+-----------+----------+-----+--------+----------+



In [ ]:
df_clean=df.dropna(subset=['product','price','quantity','order_date']).withColumn("Total Price", col("price") * col("quantity")).withColumn("order_date",to_date(col("order_date"), "dd-MM-yyyy"))

print("Cleansed Data :")

df_clean.show()

Cleansed Data :
+--------+-----------+----------+-----+--------+----------+-----------+
|order_id|customer_id|   product|price|quantity|order_date|Total Price|
+--------+-----------+----------+-----+--------+----------+-----------+
|    1001|       C001|    Laptop|70000|       1|2024-06-01|      70000|
|    1002|       C002|Smartphone|30000|       2|2024-06-02|      60000|
|    1003|       C001|     Mouse|  500|       3|2024-06-03|       1500|
|    1004|       C003|  Keyboard| 1500|       1|2024-06-04|       1500|
|    1005|       C002|   Monitor|12000|       1|2024-06-05|      12000|
|    1008|       C001|    Laptop|70000|       1|2024-06-08|      70000|
|    1010|       C007|     Mouse|  500|       2|2024-06-10|       1000|
+--------+-----------+----------+-----+--------+----------+-----------+



In [ ]:
df_clean.write.mode("overwrite").option("header", True).csv("/content/cleansed_sales")

!zip -r cleansed_sales.zip cleansed_sales
files.download("cleansed_sales.zip")

  adding: cleansed_sales/ (stored 0%)
  adding: cleansed_sales/._SUCCESS.crc (stored 0%)
  adding: cleansed_sales/.part-00000-85627b1b-e63b-4496-990f-2da7a31066b6-c000.csv.crc (stored 0%)
  adding: cleansed_sales/part-00000-85627b1b-e63b-4496-990f-2da7a31066b6-c000.csv (deflated 49%)
  adding: cleansed_sales/_SUCCESS (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_clean.groupBy("product").agg({"Total Price": "sum"}).withColumnRenamed("Sum(Total Price)", "Total Sales").orderBy("Total Sales", ascending=False).show()

+----------+-----------+
|   product|Total Sales|
+----------+-----------+
|    Laptop|     140000|
|Smartphone|      60000|
|   Monitor|      12000|
|     Mouse|       2500|
|  Keyboard|       1500|
+----------+-----------+



In [ ]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("My first Pyspark Program").getOrCreate()

print("Say Hello")

data=[("saravana", 27),("sudha", 26)]

df=spark.createDataFrame(data, schema=["name","age"])
df.printSchema()
df.show()

Say Hello
root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)

+--------+---+
|    name|age|
+--------+---+
|saravana| 27|
|   sudha| 26|
+--------+---+



In [ ]:
df.select("name").show()

df.filter(df.age == 26)

+--------+
|    name|
+--------+
|saravana|
|   sudha|
+--------+

+--------+---+
|    name|age|
+--------+---+
|saravana| 27|
|   sudha| 26|
+--------+---+



In [ ]:
df=df.withColumn("Age_plus", col("age")+5)

df.show()

+--------+---+--------+
|    name|age|Age_plus|
+--------+---+--------+
|saravana| 27|      32|
|   sudha| 26|      31|
+--------+---+--------+



In [ ]:
df.filter(df.Age_plus>31).show()

+--------+---+--------+
|    name|age|Age_plus|
+--------+---+--------+
|saravana| 27|      32|
|   sudha| 26|      31|
+--------+---+--------+

+--------+---+--------+
|    name|age|Age_plus|
+--------+---+--------+
|saravana| 27|      32|
+--------+---+--------+



In [ ]:
df.withColumnRenamed("name","Name").withColumnRenamed("age","Age").show()

+--------+---+--------+
|    Name|Age|Age_plus|
+--------+---+--------+
|saravana| 27|      32|
|   sudha| 26|      31|
+--------+---+--------+



In [ ]:
from pyspark.sql.types import StructField,StructType,StringType,IntegerType

schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True)
])
new_data = [("Raga",None,5),("sarvan",None,5)]
new_df=spark.createDataFrame(new_data,["Name","Age","Age_plus"])
new_df.show()

PySparkValueError: [CANNOT_DETERMINE_TYPE] Some of types cannot be determined after inferring.

In [ ]:
df_combined = df.union(new_df)
df_combined.show()

+--------+---+--------+
|    name|age|Age_plus|
+--------+---+--------+
|saravana| 27|      32|
|   sudha| 26|      31|
|    Raga|  0|       5|
|  sarvan|  0|       5|
+--------+---+--------+



In [ ]:
df_combined.dropna().show()

+--------+---+--------+
|    name|age|Age_plus|
+--------+---+--------+
|saravana| 27|      32|
|   sudha| 26|      31|
|    Raga|  0|       5|
|  sarvan|  0|       5|
+--------+---+--------+



In [ ]:
df_combined = df.union(new_df)
df_combined.dropna().show()

NameError: name 'df_combined' is not defined

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark=SparkSession.builder.appName("Sample").getOrCreate()
bio_data=[("saravana",27),("sudha",26)]
df=spark.createDataFrame(bio_data,schema=["name","age"])

df.show()

df.filter(df["age"]>26).show()
df.filter(col("age")>=27).show()

+--------+---+
|    name|age|
+--------+---+
|saravana| 27|
|   sudha| 26|
+--------+---+

+--------+---+
|    name|age|
+--------+---+
|saravana| 27|
+--------+---+

+--------+---+
|    name|age|
+--------+---+
|saravana| 27|
+--------+---+



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

from google.colab import files
uploaded=files.upload()

spark=SparkSession.builder.appName("Aggregations").getOrCreate()
df_oper=spark.read.option("header",True).option("inferschema", True).csv("sales.csv")

df_oper.dropna().show()

Saving sales.csv to sales (1).csv
+--------+-----------+----------+-----+--------+----------+
|order_id|customer_id|   product|price|quantity|order_date|
+--------+-----------+----------+-----+--------+----------+
|    1001|       C001|    Laptop|70000|       1|01-06-2024|
|    1002|       C002|Smartphone|30000|       2|02-06-2024|
|    1003|       C001|     Mouse|  500|       3|03-06-2024|
|    1004|       C003|  Keyboard| 1500|       1|04-06-2024|
|    1005|       C002|   Monitor|12000|       1|05-06-2024|
|    1008|       C001|    Laptop|70000|       1|08-06-2024|
|    1010|       C007|     Mouse|  500|       2|10-06-2024|
+--------+-----------+----------+-----+--------+----------+



In [ ]:
df_oper.orderBy(col("price").desc()).show()
df_oper.orderBy("price").show()

print(df_oper.count())

+--------+-----------+----------+-----+--------+----------+
|order_id|customer_id|   product|price|quantity|order_date|
+--------+-----------+----------+-----+--------+----------+
|    1001|       C001|    Laptop|70000|       1|01-06-2024|
|    1008|       C001|    Laptop|70000|       1|08-06-2024|
|    1006|       C004|      NULL|45000|       1|06-06-2024|
|    1002|       C002|Smartphone|30000|       2|02-06-2024|
|    1007|       C005|    Tablet|25000|    NULL|07-06-2024|
|    1005|       C002|   Monitor|12000|       1|05-06-2024|
|    1009|       C006|Smartwatch| 5000|       1|      NULL|
|    1004|       C003|  Keyboard| 1500|       1|04-06-2024|
|    1003|       C001|     Mouse|  500|       3|03-06-2024|
|    1010|       C007|     Mouse|  500|       2|10-06-2024|
+--------+-----------+----------+-----+--------+----------+

+--------+-----------+----------+-----+--------+----------+
|order_id|customer_id|   product|price|quantity|order_date|
+--------+-----------+----------+-----+

In [ ]:
from pyspark.sql.functions import avg,when, col
df_oper.groupBy("order_id").agg(avg("price").alias("Average Price")).show()

+--------+-------------+
|order_id|Average Price|
+--------+-------------+
|    1005|      12000.0|
|    1008|      70000.0|
|    1010|        500.0|
|    1002|      30000.0|
|    1001|      70000.0|
|    1006|      45000.0|
|    1007|      25000.0|
|    1003|        500.0|
|    1004|       1500.0|
|    1009|       5000.0|
+--------+-------------+



In [5]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("sample").getOrCreate()
nums = spark.sparkContext.parallelize([1,2,3,4])
squared = nums.map(lambda a: a*a)
print(squared.collect())

[1, 4, 9, 16]


In [ ]:
df_oper=df_oper.withColumn("customer_NAME", when(col("customer_id") == "C001", "Saravana")
  .when(col("customer_id") == "C002", "Sudha")
  .when(col("customer_id") == "C003", "Sarvan")
  .when(col("customer_id") == "C004", "Ram")
  .when(col("customer_id") == "C005", "Lav")
  .when(col("customer_id") == "C006", "Chev")
  .when(col("customer_id") == "C007", "HeyRam")).show()

+--------+-----------+----------+-----+--------+----------+-------------+
|order_id|customer_id|   product|price|quantity|order_date|customer_NAME|
+--------+-----------+----------+-----+--------+----------+-------------+
|    1001|       C001|    Laptop|70000|       1|01-06-2024|     Saravana|
|    1002|       C002|Smartphone|30000|       2|02-06-2024|        Sudha|
|    1003|       C001|     Mouse|  500|       3|03-06-2024|     Saravana|
|    1004|       C003|  Keyboard| 1500|       1|04-06-2024|       Sarvan|
|    1005|       C002|   Monitor|12000|       1|05-06-2024|        Sudha|
|    1006|       C004|      NULL|45000|       1|06-06-2024|          Ram|
|    1007|       C005|    Tablet|25000|    NULL|07-06-2024|          Lav|
|    1008|       C001|    Laptop|70000|       1|08-06-2024|     Saravana|
|    1009|       C006|Smartwatch| 5000|       1|      NULL|         Chev|
|    1010|       C007|     Mouse|  500|       2|10-06-2024|       HeyRam|
+--------+-----------+----------+-----

In [ ]:
data = [("Alice", "28"), ("Bob", "25"), ("Cathy", "30")]
df = spark.createDataFrame(data, ["name", "age"])

df.printSchema()
df.show()
# Cast 'age' to integer
df_casted = df.withColumn("age", col("age").cast("integer"))

df_casted.printSchema()
df_casted.show()

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)

+-----+---+
| name|age|
+-----+---+
|Alice| 28|
|  Bob| 25|
|Cathy| 30|
+-----+---+

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)

+-----+---+
| name|age|
+-----+---+
|Alice| 28|
|  Bob| 25|
|Cathy| 30|
+-----+---+



In [10]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("RDDHandson").getOrCreate()
sc=spark.sparkContext
data=['hello world','welcome to spark','This is a simple handson on spark']
rdd=sc.parallelize(data)
words=rdd.flatMap(lambda line: line.split(" "))
pairs=words.map(lambda words: (words,1))
counts=pairs.reduceByKey(lambda a,b : (a+b))
print(counts.collect())


[('hello', 1), ('world', 1), ('to', 1), ('welcome', 1), ('spark', 2), ('This', 1), ('is', 1), ('a', 1), ('simple', 1), ('handson', 1), ('on', 1)]


In [9]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("sample").getOrCreate()
sc=spark.sparkContext
data=['hello world','welcome to spark world']
rdd=sc.parallelize(data)
words=rdd.flatMap(lambda line: line.split(" "))
pairs=words.map(lambda word: (word,1))
group=pairs.reduceByKey(lambda a,b: (a+b))
print(group.collect())


[('hello', 1), ('world', 2), ('to', 1), ('welcome', 1), ('spark', 1)]


In [10]:
nums=sc.parallelize(['1','1','2','3','3','4','5'])
unique=nums.distinct()
unique.collect()

['3', '4', '5', '1', '2']

In [40]:
marks=sc.parallelize(['94','78','100','100','96'])
nums_int=marks.map(lambda x:int(x))
total=nums_int.reduce(lambda a,b : a+b)
avg=total/marks.count()
print(avg)

93.6


In [23]:
nums=sc.parallelize(range(1,30))
pairs=nums.map(lambda nums: ('even', 1) if (nums%2==0) else ('odd',1))

groups=pairs.reduceByKey(lambda a,b : a+b)
print(groups.collect())

[('even', 14), ('odd', 15)]


In [29]:
#finding 3 largest numbers
nums=sc.parallelize([10,20,30,60,40,50])
larg=nums.takeOrdered(3, key=lambda x: -x)
print(larg)

[60, 50]


In [28]:
nums=sc.parallelize([10,20,30,60,40,50])
larg=nums.takeOrdered(3)
print(larg)

[10, 20, 30]


In [33]:
text="This is saravanan"
chars=sc.parallelize(list(text.replace(" ","")))
count=chars.countByValue()
print(count)

defaultdict(<class 'int'>, {'T': 1, 'h': 1, 'i': 2, 's': 3, 'a': 4, 'r': 1, 'v': 1, 'n': 2})


In [39]:
text="This is saravanan"
chars=sc.parallelize(list(text.replace(" ","")))
pair=chars.map(lambda char:(char,1))
count=pair.reduceByKey(lambda a,b:a+b)
print(count.collect())

[('T', 1), ('h', 1), ('i', 2), ('s', 3), ('a', 4), ('r', 1), ('v', 1), ('n', 2)]
